## Imports

In [ ]:
import json
from http.cookies import SimpleCookie
from io import BytesIO
from pprint import pprint

import requests
import httpx
import aiohttp
from PIL import Image
from curl_cffi import requests as curl_requests
from curl_cffi.requests import AsyncSession
import matplotlib.pyplot as plt


## Constants

In [ ]:
DEFAULT_USER_AGENT = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/134.0.0.0 Safari/537.36"
)

DEFAULT_USER_HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en,zh;q=0.9,zh-CN;q=0.8",
    "User-Agent": DEFAULT_USER_AGENT,
    "Connection": "keep-alive",
}
LENGTH = 500


## Helper Functions

In [ ]:
def cookie_str_to_dict(cookie_str: str) -> dict[str, str]:
    """Convert a cookie string into a dictionary usable by requests/httpx/aiohttp."""
    cookie = SimpleCookie()
    cookie.load(cookie_str)
    return {key: morsel.value for key, morsel in cookie.items()}

def display_text_response(content: str, status_code: int, length: int = LENGTH) -> None:
    """Display HTTP text response in a readable form."""
    print(f"Status code: {status_code}")
    if not content:
        print("(empty response)")
        return

    snippet = content[:length]
    print(f"Content preview ({min(len(content), length)} chars):")
    print(snippet)
    if len(content) > length:
        print("... [truncated]")

def display_image_response(content: bytes, status_code: int) -> None:
    """Display HTTP image response inline within a Jupyter notebook."""

    print(f"Status code: {status_code}")
    if not content:
        print("(empty response)")
        return

    try:
        img = Image.open(BytesIO(content))
    except Exception as e:
        print(f"Failed to open image: {e}")
        return

    plt.imshow(img)
    plt.axis("off")
    plt.title(f"HTTP {status_code}")
    plt.show()

def display_bytes_response(content: bytes, status_code: int, length: int = LENGTH) -> None:
    """Display raw bytes response info (e.g., binary data, files)."""
    print(f"Status code: {status_code}")
    if not content:
        print("(empty response)")
        return

    preview = content[:length]
    print(f"Byte length: {len(content)} bytes")
    print(f"Content preview ({min(len(content), length)} bytes): {preview!r}")
    if len(content) > length:
        print("... [truncated]")


def display_json_response(content: bytes | str | dict, status_code: int, length: int = LENGTH) -> None:
    """Display JSON response prettily formatted."""
    print(f"Status code: {status_code}")
    if not content:
        print("(empty response)")
        return

    try:
        if isinstance(content, (bytes, str)):
            data = json.loads(content)
        else:
            data = content
        formatted = json.dumps(data, indent=2, ensure_ascii=False)
    except Exception as e:
        print(f"Failed to parse JSON: {e}")
        return

    if len(formatted) > length:
        print(f"JSON preview ({length} chars):\n{formatted[:length]}")
        print("... [truncated]")
    else:
        print("JSON content:")
        print(formatted)


## GET Requests


In [ ]:

resp_type: str = "bytes"  # "text" or "image" or "bytes" or "json"
url = "https://example.com"
headers = {
    **DEFAULT_USER_HEADERS,
    # "Referer": "https://www.google.com/",
}
params = {
    # "keyword": "example",
}
cookies_str = r"""
"""  # Optional cookie string

encoding = None
# encoding = "gbk"
# encoding = "utf-8"
http2: bool = True  # for httpx
impersonate = None  # for curl_cffi, e.g. "chrome124", "chrome", None
# impersonate = "chrome"

cookies = cookie_str_to_dict(cookies_str)


In [ ]:
# requests (sync)
resp = requests.get(url, headers=headers, params=params, cookies=cookies)

print("requests")

match resp_type:
    case "text":
        if encoding:
            resp.encoding = encoding
        display_text_response(resp.text, resp.status_code)

    case "json":
        if encoding:
            resp.encoding = encoding
        display_json_response(resp.text, resp.status_code)

    case "image":
        display_image_response(resp.content, resp.status_code)

    case "bytes":
        display_bytes_response(resp.content, resp.status_code)

    case _:
        print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# httpx (sync)
with httpx.Client(http2=http2, headers=headers, cookies=cookies) as client:
    r = client.get(url, params=params)

    print("httpx sync")

    match resp_type:
        case "text":
            if encoding:
                r.encoding = encoding
            display_text_response(r.text, r.status_code)

        case "json":
            if encoding:
                r.encoding = encoding
            display_json_response(r.text, r.status_code)

        case "image":
            display_image_response(r.content, r.status_code)

        case "bytes":
            display_bytes_response(r.content, r.status_code)

        case _:
            print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# httpx (async)
async with httpx.AsyncClient(http2=http2, headers=headers, cookies=cookies) as client:
    r = await client.get(url, params=params)

    print("httpx async")

    match resp_type:
        case "text":
            if encoding:
                r.encoding = encoding
            display_text_response(r.text, r.status_code)

        case "json":
            if encoding:
                r.encoding = encoding
            display_json_response(r.text, r.status_code)

        case "image":
            display_image_response(r.content, r.status_code)

        case "bytes":
            display_bytes_response(r.content, r.status_code)

        case _:
            print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# curl_cffi (sync)
r = curl_requests.get(url, headers=headers, cookies=cookies, impersonate=impersonate)

print("curl_cffi sync")

match resp_type:
    case "text":
        if encoding:
            r.encoding = encoding
        display_text_response(r.text, r.status_code)

    case "json":
        if encoding:
            r.encoding = encoding
        display_json_response(r.text, r.status_code)

    case "image":
        display_image_response(r.content, r.status_code)

    case "bytes":
        display_bytes_response(r.content, r.status_code)

    case _:
        print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# curl_cffi (async)
async with AsyncSession(
    impersonate=impersonate,
    headers=headers,
    cookies=cookies
) as s:
    r = await s.get(url)

    print("curl_cffi async")

    match resp_type:
        case "text":
            if encoding:
                r.encoding = encoding
            display_text_response(r.text, r.status_code)

        case "json":
            if encoding:
                r.encoding = encoding
            display_json_response(r.text, r.status_code)

        case "image":
            display_image_response(r.content, r.status_code)

        case "bytes":
            display_bytes_response(r.content, r.status_code)

        case _:
            print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# aiohttp (async)
async with aiohttp.ClientSession(headers=headers, cookies=cookies) as s:
    async with s.get(url, params=params) as r:
        print("aiohttp async")

        match resp_type:
            case "text":
                text = await r.text(encoding=encoding)
                display_text_response(text, r.status)

            case "json":
                text = await r.text(encoding=encoding)
                display_json_response(text, r.status)

            case "image":
                content = await r.read()
                display_image_response(content, r.status)

            case "bytes":
                content = await r.read()
                display_bytes_response(content, r.status)

            case _:
                print(f"Unknown resp_type: {resp_type!r}")


## POST Requests

In [ ]:

resp_type: str = "text"  # "text" or "image" or "bytes" or "json"
url = "https://example.com/post"
headers = {
    **DEFAULT_USER_HEADERS,
    # "Referer": "https://www.google.com/",
}
data = {
    # "keyword": "example",
}
params = {
    # "keyword": "example",
}
cookies_str = r"""
"""  # Optional cookie string

length = 500
encoding = None
# encoding = "gbk"
# encoding = "utf-8"
http2: bool = True  # for httpx
impersonate = None  # for curl_cffi, e.g. "chrome124", "chrome", None
# impersonate = "chrome"

cookies = cookie_str_to_dict(cookies_str)


In [ ]:
# requests (sync)
resp = requests.post(url, headers=headers, params=params, data=data, cookies=cookies)

print("requests")

match resp_type:
    case "text":
        if encoding:
            resp.encoding = encoding
        display_text_response(resp.text, resp.status_code)

    case "json":
        if encoding:
            resp.encoding = encoding
        display_json_response(resp.text, resp.status_code)

    case "image":
        display_image_response(resp.content, resp.status_code)

    case "bytes":
        display_bytes_response(resp.content, resp.status_code)

    case _:
        print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# httpx (sync)
with httpx.Client(http2=http2, headers=headers, cookies=cookies) as client:
    r = client.post(url, params=params, data=data)

    print("httpx sync")

    match resp_type:
        case "text":
            if encoding:
                r.encoding = encoding
            display_text_response(r.text, r.status_code)

        case "json":
            if encoding:
                r.encoding = encoding
            display_json_response(r.text, r.status_code)

        case "image":
            display_image_response(r.content, r.status_code)

        case "bytes":
            display_bytes_response(r.content, r.status_code)

        case _:
            print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# httpx (async)
async with httpx.AsyncClient(http2=http2, headers=headers, cookies=cookies) as client:
    r = await client.post(url, params=params, data=data)

    print("httpx async")

    match resp_type:
        case "text":
            if encoding:
                r.encoding = encoding
            display_text_response(r.text, r.status_code)

        case "json":
            if encoding:
                r.encoding = encoding
            display_json_response(r.text, r.status_code)

        case "image":
            display_image_response(r.content, r.status_code)

        case "bytes":
            display_bytes_response(r.content, r.status_code)

        case _:
            print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# curl_cffi (sync)
r = curl_requests.post(url, headers=headers, cookies=cookies, data=data, impersonate=impersonate)

print("curl_cffi sync")
display_text_response(r.text, r.status_code)

match resp_type:
    case "text":
        if encoding:
            r.encoding = encoding
        display_text_response(r.text, r.status_code)

    case "json":
        if encoding:
            r.encoding = encoding
        display_json_response(r.text, r.status_code)

    case "image":
        display_image_response(r.content, r.status_code)

    case "bytes":
        display_bytes_response(r.content, r.status_code)

    case _:
        print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# curl_cffi (async)
async with AsyncSession(impersonate=impersonate, headers=headers, cookies=cookies) as s:
    r = await s.post(url, data=data, params=params)

    print("curl_cffi async")

    match resp_type:
        case "text":
            if encoding:
                r.encoding = encoding
            display_text_response(r.text, r.status_code)

        case "json":
            if encoding:
                r.encoding = encoding
            display_json_response(r.text, r.status_code)

        case "image":
            display_image_response(r.content, r.status_code)

        case "bytes":
            display_bytes_response(r.content, r.status_code)

        case _:
            print(f"Unknown resp_type: {resp_type!r}")


In [ ]:
# aiohttp (async)
async with aiohttp.ClientSession(headers=headers, cookies=cookies) as s:
    async with s.post(url, data=data, params=params) as r:
        print("aiohttp async")

        match resp_type:
            case "text":
                text = await r.text(encoding=encoding)
                display_text_response(text, r.status)

            case "json":
                text = await r.text(encoding=encoding)
                display_json_response(text, r.status)

            case "image":
                content = await r.read()
                display_image_response(content, r.status)

            case "bytes":
                content = await r.read()
                display_bytes_response(content, r.status)

            case _:
                print(f"Unknown resp_type: {resp_type!r}")
